# 04. CODE REPRESENTATION

1. [x] введение
2. [x] CodeBERT
3. [ ] GraphCodeBERT
4. [ ] UniXCoder
5. [x] упражнение
6. [x] ссылки

# 1. Введение

Гипотеза (например, [Hindle et al - On the naturalness of software 2016](https://dl.acm.org/doi/abs/10.1145/2902362)):
>
> *Язык программирования (PL) похож на естественный язык (NL).*

Если так, то можем использовать эффективные подходы из NLP, учитывая специфику PL.

Encoder-only:
- CuBERT (Kanade et al, 2019)
- CodeBERT (Feng et al, 2020)
- GraphCodeBERT (Guo et al, 2020)
- SynCoBERT (Wang et al, 2022)

Decoder-only:
- GPT-C (Svyatkovskiy et al, 2020)
- CodeGPT (Lu et al, 2021)

Encoder-decoder:
- PLBART (Ahmad et al, 2021)
- CodeT5 (Wang et al, 2021)
- TreeBERT (Jiang et al, 2021)
- UniXcoder (Guo et al, 2022)

# 2. CodeBERT

[CodeBERT](https://arxiv.org/abs/2002.08155v4) --- бимодальная предварительно обученная модель для языков программирования и естественного языка. CodeBERT на выходе выдаёт эмбеддинги общего назначения. Эмбеддинги могут использоваться в таких задач, как поиск по коду, суммаризация кода и т. д. Модель разработана в Microsoft в 2020 году.

![](./res/04_codebert_paper.png)

## Архитектура

В основе модели CodeBERT лежит [BERT](https://arxiv.org/abs/1810.04805)(Google, 2018). Точнее, [RoBERTa](https://arxiv.org/abs/1907.11692) (Meta, 2019, отличие в процедуре формирования масок).

![](https://miro.medium.com/max/720/1*p4LFBwyHtCw_Qq9paDampA.png)

In [ ]:
import pprint
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [ ]:
nl_tokens=tokenizer.tokenize("return maximum value")
print(nl_tokens)

In [ ]:
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")
print(code_tokens)

In [ ]:
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]
print(tokens)

In [ ]:
tokens_ids=tokenizer.convert_tokens_to_ids(tokens)
print(tokens_ids)

In [ ]:
len(tokens_ids)

In [ ]:
context_embeddings=model(torch.tensor(tokens_ids)[None,:])[0]

In [ ]:
context_embeddings.shape

In [ ]:
print(context_embeddings)

## Pre-training


CodeBERT --- это бимодальная модель. Это значит, что на вход могут подаваться пары вида (NL, PL).
При этом пара преобразуется в последовательность токенов.
Для этого, каждый элемент пары: NL- и PL-часть, с помощью токенизатора ([Byte Pair Encoding](https://en.wikipedia.org/wiki/Byte_pair_encoding), BPE) превращается в последовательность токенов (сегмент).

На этапе предобучения модели на вход подаётся конкатенация двух сегментов со специальным токеном-разделителем:

$$[\texttt{CLS}], w_1, w_2, \ldots, w_n, [\texttt{SEP}], c_1, c_2, \ldots, c_m, [\texttt{EOS}].$$

- первый сегмент: текст естественного языка
- второй сегмент: исходный код

$[\texttt{CLS}]$ --- специальный токен для получение агрегированного эмбеддинга (может быть использован, например, при классификации)

На выходе имеем:
- контекстное векторное представление каждого токена (для естественного языка и кода);
- представление для токена $[\texttt{CLS}]$.


### Данные

Обучение модели CodeBERT происходит как на *бимодальных* данных (параллельные пары текст и кода), так и на *унимодальных* данных.

- из репозиториев Github
- бимодальная пара --- это отдельная функция и соответствующая документация (примерно, 2 млн)
- унимодальные данные --- это функция без парной документации (примерно, 6 млн)
- шесть языков программирования: Python, Java, JavaScript, PHP, Ruby и Go.

### Обучение

Предобучение происходит на двух задачах:
- MLM (masked language modeling), BERT, Devlin et al. (2018): восстановление токенов, бимодальные данные
- RTD (replaced token detection), ELECTRA, Clark et al. (2020): обнаружение замены токенов, унимодальные данные

**Задача MLM**

![](https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/docs/img/MLM.png)

Пусть $x = \{w, c\}$ --- входная NL-PL пара, где $w$ --- последовательность NL-токенов, а $c$ --- последовательность PL-токенов.

Сначала выбираем случайный набор позиций для NL и PL, которые необходимо замаскировать --- $m_w$ и $m_c$ соответственно. Затем заменяем выбранные позиции специальным токеном $[\texttt{MASK}]$. Как и в Devlin et al. (2018), маскируется 15% токенов из $x$ (если выбран токен, то с вероятностью 0.8 он заменяется на $[\texttt{MASK}]$, с вероятностью 0.1 заменяется на случайный токен, с вероятностью 0.1 остаётся прежним).

- для $i \in \{1, \ldots, |w|\}$ имеем $m^w_i ∼ \texttt{unif}\{1, |w|\}$ --- какие NL-токены хотим маскировать
- для $i \in \{1, \ldots, |c|\}$ имеем $w^c_m ∼ \texttt{unif}\{1, |c|\}$ --- какие PL-токены хотим маскировать
- $w^{masked} = \texttt{REPLACE}(w, m^w, [\texttt{MASK}])$
- $c^{masked} = \texttt{REPLACE}(c, m^c, [\texttt{MASK}])$
- $x = [w;c]$ --- конкатенируем

В MLM задача состоит в том, чтобы предсказать токены, которые были маскированы. Для этого используется следующая функция потерь:

$$L_{MLM}(\theta) = \sum_{i \in m^w \cup m^c} -\log p (x_i~|~w^{masked}, c^{masked}).$$

In [ ]:
from transformers import RobertaConfig, RobertaTokenizer, RobertaForMaskedLM, pipeline

model = RobertaForMaskedLM.from_pretrained("microsoft/codebert-base-mlm")
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")

In [ ]:
CODE = """
if x is not <mask>:
    x += 1"
"""
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)

outputs = fill_mask(CODE)
pp.pprint(outputs)

**Задача RTD**

Изначально, задача RTD (replaced token detection) была использована для предобучния моделей естественного языка [ELECTRA](https://arxiv.org/abs/2003.10555). Для модели CodeBERT эта задача была адаптирована для кода.
Используются как *бимодальные*, так и *унимодальные* данные. В частности, здесь есть два генератора данных:
- NL-генератор $p^{G_w}$ и
- PL-генератор $p^{G_c}$.

Оба генератора нужны для создания правдоподобных альтернатив выбранных токенов.

![](./res/04_rtd_electra.png)

Генератор --- произвольая языковая модель, генерирующая распределение на токенах. Обычно используют небольшую  модель, обучающуюся совместно с генератором.

Более формально:
- для $i \in m^w$ имеем $\hat{w}_i ∼ p^{G_w} (w_i~|~w_{masked})$ --- какие токены и на какие хотим заменить
- для $i \in m^c$ имеем $\hat{c}_i ∼ p^{G_c} (c_i~|~c_{masked})$ --- аналогично для кода
- $w^{corrupt} = \texttt{REPLACE}(w, m^w , \hat{w})$
- $c^{corrupt} = \texttt{REPLACE}(c, m^c , \hat{c})$
- $x^{corrupt} = [w^{corrupt};c^{corrupt}]$ --- конкатенируем 

Задача дискриминатора определить, является ли токен оригинальным или нет, т.е. бинарная классификация.
![](./res/04_rtd_codebert.png)

Заметим, что генератор применяется к каждой позиции во входных данных и отличается от GAN тем,
 что, если генератор выдает правильный токен, то метка этого токена "original", а не "replaced".


Функция потерь для задачи RTD:

$$L_{RTD}(\theta) = \sum_{i=1}^{|w|+|c|}(\delta(i) \log p^{D} (x^{corrupt}, i) + (1-\delta(i))(1-\log p^{D}(x^{corrupt}, i))),$$

где
- $\theta$ --- параметры,
- $p^{D}$ --- дискриминатор, предсказывающий вероятность того, что $i$-й токен является оригинальным,
- $\delta(i)$ --- индикаторная функция: $\delta(i) = 1$, если $x^{corrupt}_i = x_i$; иначе, $\delta(i) = 0.$

Существует множество различных способов реализации генераторов. Для CodeBERT используются две модели языка (NL, PL) на основе  $n$-грамм (Jurafsky, 2000) с двунаправленными контекстами. Модели были обучены на соответствующих унимодальных данных.

**Задача MLM+RTD**

Итоговая задача выглядит так:
$$ \min_{\theta}( L_{MLM}(\theta) + L_{RTD}(\theta) )$$

##  Реализация

- https://github.com/microsoft/CodeBERT
- https://huggingface.co/microsoft/graphcodebert-base

# 3. GraphCodeBERT

We present GraphCodeBERT, a pre-trained model for programming language that considers the inherent structure of code. Instead of taking syntactic-level structure of code like abstract syntax tree (AST), we use data flow in the pre-training stage, which is a semantic-level structure of code that encodes the relation of “where-the-value-comes-from” between variables. Such a semantic-level structure is less complex and does not bring an unnecessarily deep hierarchy of AST, the property of which makes the model more efficient.

#### DATA FLOW

In this section, we describe the basic concept and extraction of data flow.

Data flow is a graph that represents dependency relation between variables, in which nodes represent variables and edges represent where the value of each variable comes from. Unlike AST, data flow is same under different abstract grammars for the same source code. Such code structure provides crucial code semantic information for code understanding. Taking $v = max\_value − min\_value$ as an example, programmers do not always follow the naming conventions so that it is hard to understand the semantic of the variable. Data flow provides a way to understand the semantic of the variable $v$ to some extent, i.e. the value of $v$ comes from $max\_value$ and $min\_value$ in data flow. Besides, data flow supports the model to consider long-range dependencies induced by using the same variable or function in distant locations. Taking Figure 1 as an example, there are four variables with same name (i.e. $x^3$, $x^7$, $x^9$ and $x^{11}$) but with different semantic. The graph in the figure shows dependency relation between these variables and supports $x^{11}$ to pay more attention to $x^7$ and $x^9$ instead of $x^3$. Next, we describe how to extract data flow from a source code.

![overview](./res/05_graphcodebert_dataflow.png)

Figure 1 shows the extraction of data flow through a source code. Given a source code $C = \{c_1, c_2, ..., c_n \}$, we first parse the code into an abstract syntax tree (AST) by a standard compiler tool. The AST includes syntax information of the code and terminals (leaves) are used to identify the variable sequence, denoted as $V = \{v_1, v_2, ..., v_k\}$. We take each variable as a node of the graph and an direct edge $e = < v_i, v_j >$ from $v_i$ to $v_j$ refers that the value of $j$-th variable comes from $i$-th variable. Taking $x = expr$ as an example, edges from all variables in $expr$ to $x$ are added into the graph. We denote the set of directed edges as $E = \{e_1, e_2 , ..., e_l\}$ and the graph $G(C) = (V, E)$ is data flow used to represent dependency relation between variables of the source code $C$.

#### MODEL ARCHITECTURE

![overview](./res/05_graphcodebert_overview.png)

Figure 2 shows the model architecture of GraphCodeBERT. We follow BERT (Devlin et al., 2018) and use the multi-layer bidirectional Transformer (Vaswani et al., 2017) as the model backbone. Instead of only using source code, we also utilize paired comments to pre-train the model to support more code-related tasks involving natural language such as natural language code search (Feng et al., 2020). We further take data flow, which is a graph, as a part of the input to the model.

Given a source code $C = \{c_1, c_2, ..., c_n\}$ with its comment $W = \{w_1, w_2, ..., w_m\}$, we can obtain the corresponding data flow $G(C) = (V, E)$, where $V = \{v_1, v_2, ..., v_k\}$ is a set of variables and $E = \{e_1, e_2, ..., e_l\}$ is a set of direct edges that represent where the value of each variable comes from. We concatenate the comment, source code and the set of variables as the sequence input $X = \{[CLS], W, [SEP], C, [SEP], V \}$, where $[CLS]$ is a special token in front of three segments and $[SEP]$ is a special symbol to split two kinds of data types.

GraphCodeBERT takes the sequence $X$ as the input and then converts the sequence into input vectors $H^0$. For each token, its input vector is constructed by summing the corresponding token and position embeddings. We use a special position embedding for all variables to indicate that they are nodes of data flow. The model applies N transformer layers over the input vectors to produce contextual representations $H^n = transformer_n(H^{n−1})$, $n \in [1, N]$. Each transformer layer contains an architecturally identical transformer that applies a multi-headed self-attention operation (Vaswani et al., 2017) followed by a feed forward layer over the input $H^{n-1}$ in the $n$-th layer.

$$G^n = LN (MultiAttn (H^{n-1}) + H^{n-1})$$
$$H^n = LN (FFN (G^n) + G^n)$$

where $MultiAttn$ is a multi-headed self-attention mechanism, $FFN$ is a two layers feed forward network, and $LN$ represents a layer normalization operation. For the $n$-th transformer layer, the output $\hat{G}^n$ of a multi-headed self-attention is computed via:

$$Q_i = H^{n-1}W^Q_i$$
$$K_i = H^{n-1}W^K_i$$
$$V_i = H^{n-1}W^V_i$$
$$head_i = softmax(\frac{Q_iK^T_i}{\sqrt{d_k}} + M)V_i ~~~~~ (*)$$
$$\hat{G}^n = [head_1 ; ...; head_u]W^O_n$$

where the previous layer’s output $H^{n-1} \in R^{|X|} \times R^{d_h}$ is linearly projected to a triplet of queries, keys and values using model parameters $W^Q_i, W^K_i, W^V_i \in R^{d_h} \times R^{d_k}$, respectively. $u$ is the number of heads, $d_k$ is the dimension of a head, and $W^O_n \in R^{d_h} \times R^{d_h}$ is the model parameters. $M \in R^{|X|} \times R^{|X|}$ is a mask matrix, where $M_{ij}$ is $0$ if $i$-th token is allowed to attend $j$-th token otherwise $-\infty$.

#### GRAPH-GUIDED MASKED ATTENTION

To incorporate the graph structure into Transformer, we define a graph-guided masked attention function to filter out irrelevant signals. The attention masking function could avoid the key $k_i$ attended by the query $q_j$ by adding the attention score $q^T_jk_i$ an infinitely negative value so that the attention weight becomes zero after using a softmax function. To represent dependency relation between variables, a node-query $q_{v_i}$ is allowed to attend to a node-key $k_{v_j}$ if there is a direct edge from the node $v_j$ to the node $v_i$ (i.e. $<v_j, v_i>, i \in E$) or they are the same node (i.e. $i=j$). Otherwise, the attention is masked by adding an infinitely negative value into the attention score. To represent the relation between source code tokens and nodes of the data flow, we first define a set $E'$, where $\frac{<v_i, c_j>}{<c_j, v_i>} \in E'$ if the variable $v_i$ is identified from the source code token $c_j$. We then allow the node $q_{v_i}$ and code $k_{c_j}$ attend each other if and only if $\frac{<v_i, c_j>}{<c_j, v_i>} \in E'$. More formally, we use the following graph-guided masked attention matrix as the mask matrix $M$ in the equation ( * ):
$M_{ij} = 0$ if $q_i \in \{[CLS], [SEP]\}$ or $q_i, k_j \in W \cup C$ or $<q_i, k_j> \in E \cup E'$;
$M_{ij} = - \infty$ otherwise.

#### PRE-TRAINING TASKS

We describe three pre-training tasks used for pre-training GraphCodeBERT in this section. The first task is masked language modeling (Devlin et al., 2018) for learning representation from the source code. The second task is data flow edge prediction for learning representation from data flow, where we first mask some variables’ data flow edges and then let GraphCodeBERT predict those edges. The last task is variable-alignment across source code and data flow for aligning representation between source code and data flow, which predicts where a variable is identified from.

**Masked Language Modeling**
We follow Devlin et al. (2018) to apply masked language modeling (MLM) pre-training task. Specially, we sample randomly 15% of the tokens from the source code and paired comment. We replace them with a [MASK] token 80% of the time, with a random token 10% of the time, and leave them unchanged 10% of the time. The MLM objective is to predict original tokens of these sampled tokens, which has proven effective in previous works (Devlin et al., 2018; Liu et al., 2019; Feng et al., 2020). In particular, the model can leverage the comment context if the source code context is not sufficient to infer the masked code token, encouraging the model to align the natural language and programming language representations.

**Edge Prediction**
To learn representation from data flow, we introduce a pre-training task of data flow edges prediction. The motivation is to encourage the model to learn structure-aware representation that encodes the relation of “where-the-value-comes-from” for better code understanding. Specially, we randomly sample 20% of nodes $V_s$ in data flow, mask direct edges connecting these sampled nodes by add an infinitely negative value in the mask matrix, and then predict these masked edges $E_{mask}$. Taking the variable $x^{11}$ in Figure 2 for an example, we first mask edges $<x^7, x^{11}>$ and $<x^9, x^{11}>$ in the graph and then let the model to predict these edges. Formally, the pre-training objective of the task is calculated as Equation 7, where $E_c = V_s \times V \cup V \times V_s$ is a set of candidates for edge prediction, $\delta(e_{ij} \in E)$ is $1$ if $<v_i, v_j> \in E$ otherwise $0$, and the probability $p_{e_{ij}}$ of existing an edge from $i$-th to $j$-th node is calculated by dot product following a sigmoid function using representations of two nodes from GraphCodeBERT. To balance positive-negative ratio of examples, we sample negative and positive samples with the same number for $E_c$.

$$loss_{EdgePred} = - \sum_{e_{ij}\in E_c} [\delta(e_{ij} \in E_{mask})\log(p_{e_{ij}}) + (1-\delta(e_{ij} \in E_{mask})) \log(1-p_{e_{ij}})]~~~~~(7)$$

**Node Alignment**
To align representation between source code and data flow, we introduce a pre-training task of node alignment across source code and data flow, which is similar to data flow edge prediction. Instead of predicting edges between nodes, we predict edges between code tokens and nodes. The motivation is to encourage the model to align variables and source code according to data flow. Taking Figure 3 for an example, we first mask edges between the variable $x^{11}$ in data flow and code tokens, and then predict which code token the variable $x^{11}$ in data flow is identified from. As we can see, the model could predict that the variable $x^{11}$ is identified form the variable $x$ in the expression “return $x$” according to data flow information (i.e. the value of $x^{11}$ comes from $x^7$ or $x^9$).

![overview](./res/05_graphcodebert_node_task.png)

Specially, we randomly sample 20% nodes $V'_s$ in the graph, mask edges between code tokens and sampled nodes, and then predict masked edges $E'_{mask}$ . The pre-training objective of this task is similar to Equation 7, where $E'_c = V'_s \times C$ is a set of candidates for node alignment. Similarly, we also sample negative and positive samples with the same number for $E'_c$.

$$loss_{NodeAlign} = - \sum_{e_{ij}\in E'_c} [\delta(e_{ij} \in E'_{mask})\log(p_{e_{ij}}) + (1-\delta(e_{ij} \in E'_{mask})) \log(1-p_{e_{ij}})]$$

# 4. UniXCoder

# 5. Упражнение

Исследовать возможность использовать CodeBERT для задачи clone detection (поиск клонов).
Например, можно использовать датасет [BigCloneBench](https://github.com/clonebench/BigCloneBench)

# 6. Полезные ссылки

- [Kanade et al - CuBERT: Learning and Evaluating Contextual Embedding of Source Code](https://arxiv.org/abs/2001.00059)
- [Feng et al - CodeBERT: A Pre-trained Model for Programming and Natural Languages 2020](https://arxiv.org/abs/2002.08155)
- [Karampatsis Sutton - SCELMo: Source Code Embeddings from Language Models 2020](https://arxiv.org/abs/2004.13214)
- [Jain et al - Contrastive Code Representation Learning 2020](https://arxiv.org/abs/2007.04973)
- [Guo et al - GraphCodeBERT: Pre-training Code Representations with Data Flow 2020](https://arxiv.org/abs/2009.08366)
- [Chirkova Torshin - An Empirical Study of Transformers for Source Code 2020](https://arxiv.org/abs/2010.07987)
- [Gu et al - Multimodal representation for neural code search 2022](https://arxiv.org/abs/2107.00992)
- [Guo et al - UniXcoder: Unified Cross-Modal Pre-training for Code Representation](https://arxiv.org/abs/2203.03850)